# P3 Data Wrangling
Jessica Matsuoka

I chose Rio de Janeiro - Brazil [[1]](https://mapzen.com/data/metro-extracts/metro/rio-de-janeiro_brazil/), famous for its plentiful beaches and extravagant carnival. This city is easy to fall in love, with backdrop  of samba and funk rhythms. Some of the famous atrractions include Christ of Redeemer, Sugarloaf Mountain and Corcovado [[2]](https://en.wikipedia.org/wiki/Rio_de_Janeiro).


## **1. Data Overview**

The OSM XML file details:


    name: rio-de-janeiro_brazil.osm
    size: 369.7MB
    
A sample was generated to fasten the process of auditing. The sample OSM XML file details:

    name: rio-de-janeiro_brazil_sample.osm
    size: 74.4MB
    
This was done by running the script create_sample_osm.py, which you can find in the submitted folder.

Now, let's count the number of tags in the original file (OSMFILE). Running this script, we have:

    'node': 1,686,746 
    'nd': 2,105,068 
    'bounds': 1 
    'member': 27,152
    'tag': 657,508
    'osm': 1
    'way': 205,523 
    'relation': 4,257

In [43]:
OSMFILE = 'rio-de-janeiro_brazil.osm'
def count_tags(filename):
    tags = defaultdict(int)
    tree = ET.parse(filename)
    root = tree.getroot()
    for element in root.iter():
        tags[element.tag] += 1
    return tags

all_tags = count_tags(OSMFILE)
print all_tags

defaultdict(<type 'int'>, {'node': 1686746, 'nd': 2105068, 'bounds': 1, 'member': 27152, 'tag': 657508, 'osm': 1, 'way': 205523, 'relation': 4257})


In [44]:
SAMPLE_FILE = 'rio-de-janeiro_brazil_sample.osm'
sample_tags = count_tags(SAMPLE_FILE)
print sample_tags

defaultdict(<type 'int'>, {'node': 168675, 'tag': 64213, 'nd': 214891, 'response': 1, 'way': 20552})


In [45]:
def get_user(element):
    return element.get('uid')

def count_users(filename):
    users = set()
    for _, element in ET.iterparse(filename):
        user = get_user(element)
        if user is not None:
            users.add(user)

    return len(users)

count_users(OSMFILE)

1661

# #**2. Auditing the Data**

We've checked that 1661 users contributed to this map. We know that people sometimes make mistakes, and some entries might have errors. The script executed was "audit_rio.py". We will check for:

    - Lower case letters
    - Lower case letters with colon ":"
    - Any tag key with problematic/special characters
    - Other
    
Most of the cases,  
    
 **- Special Characters**
     Some streets, and cities have special characters such as acute accents ('), circumflexes (^), and cedilla (ç). In the header of the script, it was added the # -- coding: utf-8 -- to allow the manipulation of utf-8 Unicode encoding. Even doing this inclusion, it was necessary to add the letter 'u' before strings that were used for comparison. Some examples:

        - Niterói -> u'Niterói'
        - Praca -> u'Praça'
 
 **- Street Names**
     Some of the streets were abbreviated. Examples:
     
        - Rod. Washigton Luiz -> Rodovia Washington Luiz
        - Av das Américas -> Avenida das Américas
        
     Some streets didn't have any specification. Those names needed further research on Google Maps to get the correct value. Some examples:
     
        - alcides bezerra -> Rua Alcides Bezerra
        - Largo do Machado -> Praça Largo do Machado

 **- City Name:**
    The map of the region of Rio de Janeiro also have information about the metropolitan region, for this reason the key addr: city received a treatment to standardize the names. The cities list were based on Wikipedia [[3]](https://pt.wikipedia.org/wiki/Regi%C3%A3o_Metropolitana_do_Rio_de_Janeiro). There were spelling mistakes, special characters, street names instead of city, and lower case letters:
     
       - rio de janeiro  ->  Rio de Janeiro
       - Rio de Janeiro, ->  Rio de Janeiro 
       - Teresópoli -> Teresópolis
       - Rua Monsenhor Magaldi  -> Rio de Janeiro
       
     In this last case, I used Google Maps to know which city was "Rua Monsenhor Magaldi", and found that was in Rio de Janeiro city.

# 3. Cleaning the Data

# 4. Data Insights and Exploration


After auditing the data, we will create the SQLite database. The data was dumped into 'rio.db'. We are ready to explore Rio. The script executed was create_database.py.
We begin importing the database:

In [17]:
# Opening a connection with the database

import sqlite3

db = sqlite3.connect("rio.db")
c = db.cursor() 


In [18]:
# Number of nodes  
query = '''SELECT COUNT(*) FROM nodes ;'''
c.execute(query)

print "Number of nodes: ", c.fetchall()[0][0]

Number of nodes:  168675


In [19]:
# Number of ways
query = '''SELECT COUNT(*) FROM ways;'''
c.execute(query)

print "Number of ways: ", c.fetchall()[0][0]

Number of ways:  20552


And now, we will check the number of contributors:

In [20]:
#Number of unique users
query = '''SELECT COUNT(DISTINCT(e.uid)) 
           FROM (SELECT uid FROM nodes UNION ALL SELECT uid FROM ways) e;'''
c.execute(query)

print "Number of Unique users: ", c.fetchall()[0][0]

Number of Unique users:  950


In [21]:
from IPython.display import display
import pandas as pd

def show_details(query_input, rows, desc):
    df = pd.read_sql_query(query_input, db)
    print desc
    display(df.head(rows))

In [22]:
# Top 10 contributing users and number of contributions
query = '''SELECT e.user, COUNT(*) as num
           FROM (SELECT user FROM nodes UNION ALL SELECT user from ways) e
           GROUP BY e.user
           ORDER BY num DESC
           LIMIT 10;'''
show_details(query, 10, 'Top 10 contributing users and number of contributions')

Top 10 contributing users and number of contributions


,user,num
0,Alexandrecw,37112
1,smaprs_import,18396
2,ThiagoPv,17989
3,AlNo,15353
4,Import Rio,8503
5,Geaquinto,6803
6,Nighto,6642
7,Ricardo Mitidieri,5962
8,Thundercel,5466
9,patodiez,5272


And what about the languages used in Rio?
The top languages found in the database are pt (portuguese), en (english), es (spanish), ru (russian), bpv (indonesian)

In [23]:
#Languages used in Rio
query = '''
    SELECT 
        key, 
        COUNT(*) 
    FROM nodes_tags 
    WHERE type = 'name'
    GROUP BY 1 
    ORDER BY COUNT(*) DESC 
    LIMIT 5;'''
c.execute(query)

show_details(query, 5, 'Top 5 languages used')

Top 5 languages used


,key,COUNT(*)
0,pt,46
1,en,36
2,es,6
3,ru,4
4,bpv,2


In [24]:
# Shop types
query = '''
    SELECT 
        value, 
        COUNT(*) 
    FROM nodes_tags 
    WHERE key = 'shop'
    GROUP BY 1 
    ORDER BY COUNT(*) DESC 
    LIMIT 10
    ;'''

show_details(query, 10, 'Top 10 common shop types')

Top 10 common shop types


,value,COUNT(*)
0,clothes,30
1,supermarket,27
2,hairdresser,24
3,car_repair,22
4,bakery,17
5,car,16
6,convenience,16
7,hardware,13
8,bicycle,11
9,furniture,10


You might be hungry at this time. Let's find out about restaurants in Rio.

In [25]:
# Shop types
query = '''
    SELECT 
        COUNT(*) 
    FROM nodes_tags 
    WHERE key = 'amenity'
          and
          value = 'restaurant'
    ;'''

c.execute(query)

print "Number of restaurants: ", c.fetchall()[0][0]

Number of restaurants:  129


In [26]:
# Restaurant types
query = '''
    SELECT 
        value, 
        COUNT(*)  as num
    FROM nodes_tags 
    WHERE key = 'cuisine'
    AND id IN
        (SELECT 
            id 
        FROM nodes_tags
        WHERE key = 'amenity' 
        AND value = 'restaurant')
    GROUP BY value
    ORDER BY num DESC 
    LIMIT 10
    ;'''

show_details(query, 10, 'Top 10 common Restaurant types')

Top 10 common Restaurant types


,value,num
0,regional,9
1,pizza,8
2,japanese,4
3,steak_house,4
4,italian,3
5,seafood,3
6,vegetarian,2
7,Self-service,1
8,barbecue,1
9,brazilian,1


# 6. Data Improvement

Checking the Android App Store, I could find "OSM Contributor Mapping Tool" [[6]](http://wiki.openstreetmap.org/wiki/OSM_Contributor_Mapping_Tool). It would be interesting if this app would stimulate the registration of information while you were in that place. Another suggestion would be the recognition of contributors such as medals, and points. Hotels could encourage their customers to register information about their establishment and nearby interesting points that a guest could use during their stay, this would strengthen the collaboration in the region.
Another way to improve this number automatically is to get public data provided by Correios WebServices [[7]](https://www.correios.com.br/para-sua-empresa/logistica-integrada/correioslog-comercio-eletronico/pdf/CorreiosWebServicePadresTcnicosdeComunicaodoServioefulfillmentv1_1.pdf). Street names, accessibility information, tourist attractions, community centers, and other public spaces could be added to the dataset. One difficulty would be dealing with street names or abbreviation inconsistencies between Correios and OSM data. However, this could be overcome with string handling scripts and a human auditing the inputted data.

# 7. Conclusion

A significant part of Rio de Janeiro is not well formatted as we saw in the auditing phase. The absence of contributions of some streets makes this project with no statistical value and does not represent the real Rio de Janeiro.
Despite this obstacle, the map met the objective of the analysis and created the opportunity for a good learning of data wrangling using python and SQL.
                                

## References:

 - [1] (https://mapzen.com/data/metro-extracts/metro/rio-de-janeiro_brazil/)
 - [2] (https://en.wikipedia.org/wiki/Rio_de_Janeiro)
 - [3] (https://pt.wikipedia.org/wiki/Regi%C3%A3o_Metropolitana_do_Rio_de_Janeiro)
 - [4] (https://stackoverflow.com/questions/21808657/what-is-a-unicode-string)
 - [5] (https://stackoverflow.com/questions/25692293/inserting-a-link-to-a-webpage-in-an-ipython-notebook)
 - [6] (http://wiki.openstreetmap.org/wiki/OSM_Contributor_Mapping_Tool)
 - [7] (https://www.correios.com.br/para-sua-empresa/logistica-integrada/correioslog-comercio-eletronico/pdf/CorreiosWebServicePadresTcnicosdeComunicaodoServioefulfillmentv1_1.pdf)